# Matching


Background:
* 7 binary question: 2^7 = 128 possible buckets.
* MCT: Make groups based on how many questions were agreed / disagreed on.  List matches with 7, 6, 5, 4 ... differences.
* For 1-1: No labels / parties.  Just differences in 7 questions.
* Optimal distance: 4 different answers.  NOT interested in matches with <3 differences.  6-7 OK, but better to be in a group with some in-between people.
* Groups: Should have at least 2 representing the 'right' and 2 representing the 'left'.  Groups can be 5 or 6 ps, never 3 or 4.  Probably the best place for people who fall in the middle (why?)
* Potentially: First assign groups of 5, until remaining pool is ~50/50.  


In [ ]:
import numpy as np 
from scipy.spatial import distance
from scipy.spatial import distance_matrix
import time
from collections import defaultdict

IDEAL_DISTANCE = 4  # Ideal distance between pairs
MIN_DISTANCE = 3  # Min difference between pairs

def strkey(arr):
    '''Create string key for each bucket, ie "0010100".'''
    return ''.join([str(i) for i in arr])

def bucket_samples(X):
    '''Bucket each sample by strkey of responses.'''
    buckets = defaultdict(list)
    for i, x in enumerate(X):
        buckets[strkey(x)].append(i)
    return buckets

def isempty(bucketlist):
    '''Check if all buckets are empty (all ps matched).'''
    for bucket in bucketlist:
        if not bucket.isempty():
            return False
    return True

In [ ]:
'''
Bucket samples by answer patterns.
Pair up samples from largest buckets first to optimize diversity.
'''
def random_pairs(buckets):
    '''Randomly pair participants in these buckets.'''
    ps = [p for k in buckets for p in buckets[k]]
    pairs = [ps[i:i+2] for i in range(0, len(ps), 2)]
    return pairs

def pairs_at_threshold(D, buckets, bucket_idx, threshold):
    '''Find pairs for given difference threshold, largest buckets first.'''
    if threshold == 0:
        return random_pairs(buckets)
    sorted_buckets = [k for k in buckets if buckets[k]]  # All non-empty buckets
    num_ps = sum([len(buckets[b]) for b in sorted_buckets])  # Total number of participants
    pairs = []  # Pairs matched
    skip_buckets = set()  # Buckets with no remaining solutions
    def finished():
        '''Helper fn for end conditions.'''
        if len(pairs)*2 >= num_ps - 1:
            return True
        if len(sorted_buckets) - len(skip_buckets) <= 1:
            return True
        return False
    while not finished():
        sorted_buckets = sorted(  # Sort remaining buckets
            [s for s in sorted_buckets if s not in skip_buckets and buckets[s]],
            key=lambda k:len(buckets[k]), reverse=True)
        k1 = sorted_buckets[0]
        k2 = None
        for bk in sorted_buckets[1:]:
            if D[bucket_idx[k1]][bucket_idx[bk]] >= threshold:
                k2 = bk
                break
        if not k2:  # No solution at this threshold
            skip_buckets.add(k1)
        else:
            # matchcount = min(1 + len(buckets[k1]) - len(buckets[sorted_buckets[1]]), len(buckets[k2]))
            # for i in range(matchcount):
            pairs.append([buckets[k1].pop(), buckets[k2].pop()])
    return pairs


def bucketsort(X):
    buckets = bucket_samples(X)
    bucket_keys = list(buckets.keys())
    bucket_idx = {k:i for i,k in enumerate(bucket_keys)}
    bucket_vectors = [X[buckets[bk][0]] for bk in bucket_keys]
    D = (num_questions * distance.cdist(bucket_vectors, bucket_vectors, 'hamming')).astype(int)

    pairs = []  # Conversation pairs
    # TODO: Smaller thresholds earlier gives way more "fit" pairs overall
    thresholds = [IDEAL_DISTANCE]+[i for i in range(MIN_DISTANCE, IDEAL_DISTANCE)]+[i for i in range(MIN_DISTANCE - 1, -1, -1)]
    # thresholds = [MIN_DISTANCE]+[i for i in range(MIN_DISTANCE - 1, -1, -1)]
    for threshold in thresholds:
        if len(pairs)*2 < len(X):
            new_pairs = pairs_at_threshold(D, buckets, bucket_idx, threshold)
            # print("Threshold %d: %d" % (threshold, len(new_pairs)))
            pairs += new_pairs
    return pairs

def count_unfit(X, pairs, threshold=3):
    num_questions = len(X[0])
    sample_pairs = [[X[i], X[j]] for (i, j) in pairs]
    distances = [num_questions * distance.hamming(a, b) for (a, b) in sample_pairs]
    unfit_pairs = [d for d in distances if d < threshold]
    return len(unfit_pairs)

def run_experiment(X, matching_fn):
    s = 'X: %d samples, %d questions' % (len(X), len(X[0]))
    start = time.time()
    pairs = matching_fn(X)
    end = time.time()
    unfit_count = count_unfit(X, pairs, MIN_DISTANCE)
    s += "\t%.02f seconds" % (end-start)
    s += "\t%.01f%% (%d pairs) below min" % (100.*2*unfit_count/len(X), unfit_count)
    print(s)
    return pairs

In [ ]:
"""
Test with random deviations from 0000000.
"""
p = .3  # Probability of answering 1 vs 0 (.5 for even)
num_questions = 7  # Number of questions in survey

for num_samples in [500, 1000, 2000, 10000]:
    X = np.random.binomial(1, p, size=(num_samples, num_questions)).astype(int)
    run_experiment(X, bucketsort)

In [ ]:
"""
Simulate R vs D.  Assume 0s are one side, 1s are the other.  Someone from either side has r_percent chance of swapping a given answer.
"""
for num_samples in [1000, 2000, 10000, 20000]:
    p = .15  # Probability of answering along party lines
    r_percent = .3  # Probability of aligning R
    r_rows = int(r_percent * num_samples)  # Number of R rows
    X = np.random.binomial(1, p, size=(num_samples, num_questions)).astype(int)
    X[:r_rows] = 1 - X[:r_rows]
    pairs = run_experiment(X, bucketsort)

# Notes

In [ ]:
# # Generating random binary vectors.  P is probability of 1 vs 0.
# np.random.binomial(1, p, size=(num_samples, num_questions))  # Binary vectors

# # Random floats
# np.random.rand(num_samples, num_questions)  # Random

# # Random uniform between -10, 10
# np.random.uniform(-10,10,(num_samples, num_questions))  # Uniform

# Old Versions

In [ ]:
"""
V1: Pairs only, center-out.
"""
def center_out(X):
    def pick_col(row, picked):
        '''Internal fn for picking match.'''
        def score(v):
            '''Internal scoring metric.'''
            return abs(v-IDEAL_DISTANCE)
        choice = None
        for i, v in sorted(enumerate(row), key=lambda x: x[1], reverse=True):
            if i in picked:
                continue
            if not choice:
                choice = i
            if v < MIN_DISTANCE:
                break
            if score(v) < score(row[choice]):
                choice = i
                # print("trade %d(%d) for %d(%d)" % (choice, row[choice], i, v) )
        # if row[choice] < MIN_DISTANCE:
        #     raise Exception("No values less than min distance")
        return choice
    D = (num_questions * distance.cdist(X, X, 'hamming')).astype(int)
    picked = set()
    pairs = []
    mean_distances = np.mean(D, 1)
    for i in np.argsort(mean_distances):
        if i in picked:
            continue
        j = pick_col(D[i], picked)
        pairs.append((i,j))
        picked.add(i)
        picked.add(j)
    return pairs, D

# Utility functions
def count_unfit(pairs, D):
    unfit_pairs = [p for p in pairs if D[p[0]][p[1]] < MIN_DISTANCE]
    return len(unfit_pairs)

def run_co_experiment(X, matching_fn):
    s = 'X: %d samples, %d questions' % (len(X), len(X[0]))
    start = time.time()
    pairs, D = matching_fn(X)
    end = time.time()
    s += "\t%.02f seconds" % (end-start)
    unfit_count = count_unfit(pairs, D)
    s += "\t%.01f%% (%d pairs) below min" % (100.*unfit_count/len(X), unfit_count)
    print(s)

"""
Test with random deviations from 0000000.
"""
p = .2  # Probability of answering 1 vs 0 (.5 for even)
num_questions = 7  # Number of questions in survey
for num_samples in  [500, 1000, 2000, 5000]:  #, 10000]:
    X = np.random.binomial(1, p, size=(num_samples, num_questions))
    run_co_experiment(X, center_out)

In [ ]:
"""
V2: Bucketsort, largest first.
"""
def bucketsort_largest_first(X):
    buckets = bucket_samples(X)
    bucket_keys = list(buckets.keys())
    bucket_idx = {k:i for i,k in enumerate(bucket_keys)}
    bucket_vectors = [X[buckets[bk][0]] for bk in bucket_keys]
    D = (num_questions * distance.cdist(bucket_vectors, bucket_vectors, 'hamming')).astype(int)

    pairs = []  # Conversation pairs
    paired = set()  # Set of paired ps, for fast counting
    unpaired = set()  # Set of unpaired ps, for fast counting

    # Sort buckets by size, largest first
    sorted_buckets = sorted(buckets, key=lambda k:len(buckets[k]), reverse=True)

    # TODO: Consider Dynamic Thresholding: [4,5,6,7, 3] (see below)
    while len(paired) + len(unpaired) < len(X):
        # TODO: Replace with sorted_buckets and time difference
        sorted_buckets = sorted(sorted_buckets, key=lambda k:len(buckets[k]), reverse=True)
        k1 = sorted_buckets[0]
        k2 = None

        for bk in sorted_buckets[1:]:
            if not buckets[bk]:
                continue
            if D[bucket_idx[k1]][bucket_idx[bk]] >= MIN_DISTANCE:
                k2 = bk
                break
        if not k2:
            # print("No match - k%s (%d left) - %.01f%% paired"%(k1, len(buckets[k1]), 100.*len(paired)/len(X)))
            while buckets[k1]:
                unpaired.add(buckets[k1].pop())
        else:
            p1 = buckets[k1].pop()
            p2 = buckets[k2].pop()
            pairs.append([p1, p2])
            paired.add(p1)
            paired.add(p2)
    return pairs, D, unpaired

def run_experiment(X, matching_fn):
    s = 'X: %d samples, %d questions' % (len(X), len(X[0]))
    start = time.time()
    pairs, D, unpaired = matching_fn(X)
    end = time.time()
    s += "\t%.02f seconds" % (end-start)
    s += "\t%.01f%% (%d pairs) below min" % ((100.*len(unpaired)/2)/len(X), len(unpaired)/2)
    print(s)
    return pairs, D, unpaired

"""
Test with random deviations from 0000000.
"""
p = .2  # Probability of answering 1 vs 0 (.5 for even)
num_questions = 7  # Number of questions in survey

for num_samples in  [500, 1000, 2000, 10000]:
    X = np.random.binomial(1, p, size=(num_samples, num_questions)).astype(int)
    run_experiment(X, bucketsort_largest_first)

"""
Simulate R vs D.  Assume 0s are one side, 1s are the other.  Someone from either side has r_percent chance of swapping a given answer.
"""
r_percent = .20  # Probability of aligning R
r_rows = int(r_percent * num_samples)  # Number of R rows
p = .15  # Probability of answering along party lines
X = np.random.binomial(1, p, size=(num_samples, num_questions)).astype(int)
X[:r_rows] = 1 - X[:r_rows]
pairs, D, unpaired = run_experiment(X, bucketsort_largest_first)